# FS ETo Components (FS1 and FS2) Example

In [54]:

import ee
import openet.refetgee
import openet.refetgee.units as units

ee.Initialize()


### Load GRIDMET for a single date

In [55]:
gridmet_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')\
    .filterDate('2019-04-01', '2019-04-02')
    
gridmet_img = ee.Image(ee.ImageCollection(gridmet_coll.first()))
gridmet_img = ee.Image('IDAHO_EPSCOR/GRIDMET/20190801')

### Ancillary data and test point

In [56]:
test_pnt = ee.Geometry.Point(-119.01, 39.01);

# Project latitude array to match GRIDMET elevation grid exactly
elev_img = ee.Image("projects/climate-engine/gridmet/elevation")
lat_img = ee.Image.pixelLonLat().select('latitude')\
    .reproject('EPSG:4326', elev_img.projection().getInfo()['transform'])
    
# Set the output crs and crsTransform to match the GRIDMET images
crs = gridmet_img.projection().getInfo()['wkt']
geo = gridmet_img.projection().getInfo()['transform']

### Precomputed ETo

In [57]:
eto = gridmet_img.select('eto').reduceRegion(
    ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(eto.getInfo()['eto'])

7.099999904632568


### Compute FS1 for a single image

In [58]:
print(openet.refetgee.Daily.gridmet(gridmet_img, elev=elev_img, lat=lat_img).eto_fs1\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['eto_fs1'])

3.5710874773859875


### Compute FS2 for a single image

In [59]:
print(openet.refetgee.Daily.gridmet(gridmet_img, elev=elev_img, lat=lat_img).eto_fs2\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['eto_fs2'])

3.5242705617140135


### Compute ETo using FS components (FS1 + FS2)

In [60]:
fs1 = openet.refetgee.Daily.gridmet(gridmet_img, elev=elev_img, lat=lat_img).eto_fs1\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['eto_fs1']

fs2 = openet.refetgee.Daily.gridmet(gridmet_img, elev=elev_img, lat=lat_img).eto_fs2\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['eto_fs2']

eto = fs1+fs2
print(eto)

7.095358039100001
